### CARGAR DEPENDENCIAS

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
import torch.optim as optim
import numpy as np
from model import SimpleRNN, RacingDataset
import os
import torchvision.transforms.functional as F
from torchvision import transforms, models
import time
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import random

### DEFINICION DE PARAMETROS

In [14]:
#pretrained_cnn = models.efficientnet_v2_s(weights='IMAGENET1K_V1') 
pretrained_cnn = models.efficientnet_b0(weights='IMAGENET1K_V1') 
hidden_size = 256 # Número de neuronas en la capa oculta 512 usado por Iker
output_size = 9 # Número de clases (W, A, S, D, WA, WD, SA, SD, NONE)
input_size = (3, 224, 224)  # 16:9 ratio
num_layers = 1
dropout = 0 # Se necesita num_layers > 1 para que el dropout tenga efecto
bias = True
cnn_train = True # Si se desea entrenar la parte CNN 

seq_len = 5 # Número de imágenes a considerar en la secuencia
batch_size = 16 # Número de secuencias a considerar en paralelo
num_epochs = 30 # Número de veces que se recorrerá el dataset
learning_rate = 0.001 

# Parámetros del early stopping
patience = 5  # Tolerancia de 5 epochs sin mejora
best_val_loss = float('inf')  # Inicia con el peor valor posible
early_stop_counter = 0  # Contador de epochs sin mejora

# Definir ponderaciones para las etiquetas (ejemplo para 9 etiquetas)
weights = [ 1.0, # -
            2.0, # A
            2.0, # D
            0.8, # W
            2.0, # S
            1.0, # WA
            1.0, # SA
            3, # WD
            1.0] # SD


# Definir las rutas de los directorios de datos y de guardado de modelos
train_data_dir = "./datasets/train_dataset"
validation_data_dir = "./datasets/validation_dataset"
save_dir = "./trained_models"

model_name = f"Oval_Model_3_EffNet_b0_CNNGRAD_{hidden_size}_epoch"

os.makedirs(save_dir, exist_ok=True)

# Definir el escritor de TensorBoard para visualización
writer = SummaryWriter(log_dir="./runs/" + model_name) 

### INICIAR MODELO Y CARGAR DATOS

In [15]:
# Inicializar el modelo
model = SimpleRNN(pretrained_cnn, hidden_size, output_size, input_size, num_layers, dropout, bias)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Usando CUDA" if torch.cuda.is_available() else "USANDO CPU")

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Cambia el tamaño de las imágenes a (height x width)
    transforms.ToTensor(),         # Convierte las imágenes a tensores
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizar la imagen
])

# Cargar los datasets
train_dataset = RacingDataset(data_dir= train_data_dir, seq_len= seq_len, transform=transform)
test_dataset = RacingDataset(data_dir= validation_data_dir, seq_len= seq_len, transform=transform)

# Crear los dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Optimización y función de pérdida
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = CrossEntropyLoss(weight=torch.tensor(weights).to(device))
#criterion = nn.CrossEntropyLoss()

Usando CUDA


### ENTRENAMIENTO

In [16]:
print("Iniciando entrenamiento...")

start_time = time.time()    # Tiempo de inicio del entrenamiento

# Ciclo de entrenamiento
for epoch in range(num_epochs): 
    
    # Iniciar buffer de secuencia
    sequence_buffer_train = []
    sequence_buffer_validation = []

    predictions_list = [] 
    labels_list = [] 
    T_o_F_list = []
    
    model.train() # Establecer el modo de entrenamiento
    running_loss = 0.0
    try:
        for i, (images, labels) in enumerate(train_loader):

            print(f"Trabajando en Epoch {epoch+1}. Progreso: {(i+1)/len(train_loader)*100:.2f}%", end='\r')
       

            input_sequence = images # (seq_len, batch_size, channels, height, width)
            
            input_sequence, labels = input_sequence.to(device), labels.to(device) # Mover los datos al dispositivo
            
            optimizer.zero_grad()
            
            # Habilitar precisión mixta
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(input_sequence)
                loss = criterion(outputs, labels) # Solo se considera la última etiqueta de la secuencia
            
            # Backpropagation y optimización
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

            # Registrar las predicciones para la matriz de confusión
            _, predicted = torch.max(outputs, 1)
            predictions_list.append(predicted.cpu().numpy())
            labels_list.append(labels.cpu().numpy())
            T_o_F_list.append(labels.cpu().numpy() == predicted.cpu().numpy())
        

        # Registrar la pérdida en TensorBoard
        writer.add_scalar('Loss/train', running_loss / len(train_loader), epoch)

        # Guardar el modelo después de cada epoch
        torch.save(model.state_dict(), os.path.join(save_dir, f'{model_name}_{epoch+1}.pth'))

        print("\nValidando modelo...")


        # Validación en el conjunto de datos de prueba
        model.eval()  # Establecer el modo de evaluación
        test_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for i, (images, labels) in enumerate(validation_loader):
                
                print(f"Validando en Epoch {epoch+1}. Progreso: {(i+1)/len(validation_loader)*100:.2f}%", end='\r')

                # Apilar las imágenes en una sola dimensión
                input_test_sequence = images # (seq_len, batch_size, channels, height, width)

                input_test_sequence, labels = input_test_sequence.to(device), labels.to(device) # Mover los datos al dispositivo

                with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                    outputs = model(input_test_sequence)
                    loss = criterion(outputs, labels)
                    test_loss += loss.item()

                    # Predicción y cálculo de precisión
                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        val_loss = test_loss / len(validation_loader)
        writer.add_scalar('Loss/test', val_loss, epoch)
        writer.add_scalar('Accuracy/test', val_accuracy, epoch)

        # Comprobar si early stopping es necesario
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0  # Restablecer el contador
            #torch.save(model.state_dict(), os.path.join(save_dir, f'{model_name}_{epoch+1}_best.pth'))  # Guardar el mejor modelo
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping triggered after {patience} epochs without improvement.")
                break

        end_time = time.time() - start_time  # Tiempo de finalización del epoch
        # Convertir end_time a formato hh:mm:ss
        end_time = time.strftime("%H:%M:%S", time.gmtime(end_time))

        print(f'\nEpoch [{epoch+1}/{num_epochs}], '
            f'Train Loss: {running_loss/len(train_loader):.4f}, '
            f'Val Loss: {test_loss/len(validation_loader):.4f}, '
            f'Val Accuracy: {val_accuracy:.2f}% '
            f'Time: {end_time}')
    except KeyboardInterrupt:
        print("\nEntrenamiento interrumpido.")
        break
print("Entrenamiento completado.")

writer.close()  # Cerrar el escritor de TensorBoard

Iniciando entrenamiento...
Trabajando en Epoch 1. Progreso: 100.00%
Validando modelo...
Validando en Epoch 1. Progreso: 100.00%
Epoch [1/30], Train Loss: 0.9079, Val Loss: 0.5274, Val Accuracy: 86.59% Time: 00:03:41
Trabajando en Epoch 2. Progreso: 100.00%
Validando modelo...
Validando en Epoch 2. Progreso: 100.00%
Epoch [2/30], Train Loss: 0.7468, Val Loss: 0.4477, Val Accuracy: 84.08% Time: 00:07:09
Trabajando en Epoch 3. Progreso: 100.00%
Validando modelo...
Validando en Epoch 3. Progreso: 100.00%
Epoch [3/30], Train Loss: 0.6909, Val Loss: 0.4897, Val Accuracy: 82.68% Time: 00:10:39
Trabajando en Epoch 4. Progreso: 100.00%
Validando modelo...
Validando en Epoch 4. Progreso: 100.00%
Epoch [4/30], Train Loss: 0.6495, Val Loss: 0.3850, Val Accuracy: 82.68% Time: 00:14:09
Trabajando en Epoch 5. Progreso: 100.00%
Validando modelo...
Validando en Epoch 5. Progreso: 100.00%
Epoch [5/30], Train Loss: 0.6134, Val Loss: 0.4566, Val Accuracy: 78.77% Time: 00:17:40
Trabajando en Epoch 6. Progr